In [129]:
%load_ext autoreload
%autoreload 2

import requests
import json


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
#!pip install python-dotenv

# Api Key

In [131]:
#Hi yall, so for this we need riot api keys. They have 24 hour refreshes but to be extra safe put them in a .env file

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('riot_api_key')

if api_key == None:
    raise Exception(
        "Go to https://developer.riotgames.com/ and get an api key set up"
    )
else:
    print("looking good")

looking good


Read https://developer.riotgames.com/docs/portal#web-apis_api-keys for key info on api keys

# Riotwatcher

Using riotwatcher for a library to handle the api, see https://github.com/pseudonym117/Riot-Watcher

In [132]:
!pip install riotwatcher


In [133]:
# Example code from https://github.com/pseudonym117/Riot-Watcher for debugging purporses
from riotwatcher import LolWatcher, ApiError

lol_watcher = LolWatcher(api_key)

my_region = 'na1'

me = lol_watcher.summoner.by_name(my_region, 'pseudonym117')
print(me)

# all objects are returned (by default) as a dict
# lets see if i got diamond yet (i probably didnt)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

# First we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']

# Lets get some champions
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
print(current_champ_list)

# For Riot's API, the 404 status code indicates that the requested data wasn't found and
# should be expected to occur in normal operation, as in the case of a an
# invalid summoner name, match ID, etc.
#
# The 429 status code indicates that the user has sent too many requests
# in a given amount of time ("rate limiting").

try:
    response = lol_watcher.summoner.by_name(my_region, 'this_is_probably_not_anyones_summoner_name')
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('Summoner with that ridiculous name not found.')
    else:
        raise

{'id': 'dD3xFaEaFlbon9FZ6ljyBU7pXSbLV3FD36RwtK8kaC8h_-Y', 'accountId': '2AmPLTJbOTz48G33AO-dPUcXUEtRh5FdJ3LTMB9Jr9Xt2w', 'puuid': 'A6VIZHINPBHNzS1W3Yw0LltoZbJOrDIJgUEaZT7Pi2aC3TJYbv_TRQM4ywJnQflgwZbYCESO66ZAAg', 'name': 'pseudonym117', 'profileIconId': 539, 'revisionDate': 1707204724257, 'summonerLevel': 190}
[]
{'type': 'champion', 'format': 'standAloneComplex', 'version': '14.3.1', 'data': {'Aatrox': {'version': '14.3.1', 'id': 'Aatrox', 'key': '266', 'name': 'Aatrox', 'title': 'the Darkin Blade', 'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...', 'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4}, 'image': {'full': 'Aatrox.png', 'sprite': 'champion0.png', 'group': 'champion', 'x': 0, 'y': 0, 'w': 48, 'h': 48}, 'tags': ['Fighter', 'Tank'], 'partype': 'Blood

Above should show some of the example codes with the last value should be "Summoner with that ridiculous name not found". If you got an error, **refresh the api key**.

If above looks good move on to the next step

# Exploration of an example timeline!

Goal is to find the last game of the player **listed in our enviorment file** last game and the key events of that last game. 

In [134]:
player = os.getenv('player_name')

if player == None:
    raise Exception(
        "Add your riot account name to enviroment file"
    )
else:
    print("looking good")

looking good


In [135]:
# Get player name
player_data = lol_watcher.summoner.by_name(my_region, player)
print(player_data.keys())
match_ids = lol_watcher.match.matchlist_by_puuid(my_region, player_data['puuid'])
match_ids

dict_keys(['id', 'accountId', 'puuid', 'name', 'profileIconId', 'revisionDate', 'summonerLevel'])


['NA1_4913902543',
 'NA1_4913883766',
 'NA1_4913080900',
 'NA1_4913051913',
 'NA1_4909089736',
 'NA1_4909066231',
 'NA1_4909027717',
 'NA1_4908999551',
 'NA1_4907368584',
 'NA1_4907360146',
 'NA1_4907346997',
 'NA1_4907326575',
 'NA1_4907312646',
 'NA1_4907293384',
 'NA1_4906520978',
 'NA1_4906502148',
 'NA1_4906497820',
 'NA1_4906460448',
 'NA1_4906432595',
 'NA1_4906408509']

Notes
- Riot only saves last twenty games
- If we want to go through random matches, we can follow a simliar approch found at https://github.com/christophM/LolCrawler for slowing opening up diffrent matches to study
- Some matches may be just ARAM, maybe some are rift matches
- We will need to keep this in mind

How do we know if a match is an aram match or a rift match?

In [136]:
match_meta = lol_watcher.match.by_id(my_region, match_ids[0])
match_meta

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4913902543',
  'participants': ['_FznSp7W4096_tEkwFTiAmPJerwuyY9aqcMJ9CrvmeBIX59gZLPsMPb4iRdMV6DkUsIS6SeVBBc8Cw',
   'zhkSsv-0jE9dKvWh6h3xTO3UH6KIafKkI-v5E7zQjcKZWmnnzFyH415Ev-G0a1bKWo9o91IEKBumGQ',
   '8ZwlZrtP-UWTq82MS34VbSvE3opxX6dD6AFwV0VUw29ovgRGwvSmarCxljvP1qzBSz6vmvXpFVAvxQ',
   'iyNh3kBlX5g3nTPqt03REuWavBhnySCx6Ims9-nIR5C9bOf2lJa--qFlOVp-OV-EsSrwPOVJXIEstA',
   'EgKQP7KwzGTALtoAkGdy4GOi1a9yNx_JqPqQJkWD75lKm80qFB6BmPL6-ebCb4Et6q1zK3o0hH8Y2w',
   '0u5cOnC8z_gcVpDpG5OzbaiAF2-765y0yjZMseSrcHYkpg0aKV3n7rgAUG1nrHRV2E3NIYaHTa5pxg',
   '-FYL436Q4NShi_V3LqLth92pviFCqpn3Gij_vA5dnscpBwJe4B7IJSixd3jV8uOrG_Hwbru26oZM5Q',
   'zyr8N8v8UDXhe8s5kaCRh4_T2J-et-HZ3VxqQZfKt3NEavxHmbAGKvU23Gtse4wrZf2GneblWy2KFQ',
   'jG-sRLtdoqVGEym1P4ASyuqx1e13Mb1QT8OqxBz1iqbvTqyRdpRYOcGzhC0-29qsqOPNd_ZRXuiw1w',
   'GVTQeUq3T8ClMUNJNTwrekNiL_89wfaUjjlFFQI2HvU5lK5j02X8mHLY8Ar-WzyLosVyaHv8yo5RWA']},
 'info': {'gameCreation': 1707196778641,
  'gameDuration': 1033,
  

In [137]:
match_meta["info"]["gameMode"]

'ARAM'

:)

### Time for timeline

In [180]:
timeline = lol_watcher.match.timeline_by_match(my_region, match_ids[0])
timeline

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4913902543',
  'participants': ['_FznSp7W4096_tEkwFTiAmPJerwuyY9aqcMJ9CrvmeBIX59gZLPsMPb4iRdMV6DkUsIS6SeVBBc8Cw',
   'zhkSsv-0jE9dKvWh6h3xTO3UH6KIafKkI-v5E7zQjcKZWmnnzFyH415Ev-G0a1bKWo9o91IEKBumGQ',
   '8ZwlZrtP-UWTq82MS34VbSvE3opxX6dD6AFwV0VUw29ovgRGwvSmarCxljvP1qzBSz6vmvXpFVAvxQ',
   'iyNh3kBlX5g3nTPqt03REuWavBhnySCx6Ims9-nIR5C9bOf2lJa--qFlOVp-OV-EsSrwPOVJXIEstA',
   'EgKQP7KwzGTALtoAkGdy4GOi1a9yNx_JqPqQJkWD75lKm80qFB6BmPL6-ebCb4Et6q1zK3o0hH8Y2w',
   '0u5cOnC8z_gcVpDpG5OzbaiAF2-765y0yjZMseSrcHYkpg0aKV3n7rgAUG1nrHRV2E3NIYaHTa5pxg',
   '-FYL436Q4NShi_V3LqLth92pviFCqpn3Gij_vA5dnscpBwJe4B7IJSixd3jV8uOrG_Hwbru26oZM5Q',
   'zyr8N8v8UDXhe8s5kaCRh4_T2J-et-HZ3VxqQZfKt3NEavxHmbAGKvU23Gtse4wrZf2GneblWy2KFQ',
   'jG-sRLtdoqVGEym1P4ASyuqx1e13Mb1QT8OqxBz1iqbvTqyRdpRYOcGzhC0-29qsqOPNd_ZRXuiw1w',
   'GVTQeUq3T8ClMUNJNTwrekNiL_89wfaUjjlFFQI2HvU5lK5j02X8mHLY8Ar-WzyLosVyaHv8yo5RWA']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realTi

### This is a lot, gotta study the keys

In [139]:
timeline.keys()

dict_keys(['metadata', 'info'])

In [140]:
timeline["metadata"]

{'dataVersion': '2',
 'matchId': 'NA1_4913902543',
 'participants': ['_FznSp7W4096_tEkwFTiAmPJerwuyY9aqcMJ9CrvmeBIX59gZLPsMPb4iRdMV6DkUsIS6SeVBBc8Cw',
  'zhkSsv-0jE9dKvWh6h3xTO3UH6KIafKkI-v5E7zQjcKZWmnnzFyH415Ev-G0a1bKWo9o91IEKBumGQ',
  '8ZwlZrtP-UWTq82MS34VbSvE3opxX6dD6AFwV0VUw29ovgRGwvSmarCxljvP1qzBSz6vmvXpFVAvxQ',
  'iyNh3kBlX5g3nTPqt03REuWavBhnySCx6Ims9-nIR5C9bOf2lJa--qFlOVp-OV-EsSrwPOVJXIEstA',
  'EgKQP7KwzGTALtoAkGdy4GOi1a9yNx_JqPqQJkWD75lKm80qFB6BmPL6-ebCb4Et6q1zK3o0hH8Y2w',
  '0u5cOnC8z_gcVpDpG5OzbaiAF2-765y0yjZMseSrcHYkpg0aKV3n7rgAUG1nrHRV2E3NIYaHTa5pxg',
  '-FYL436Q4NShi_V3LqLth92pviFCqpn3Gij_vA5dnscpBwJe4B7IJSixd3jV8uOrG_Hwbru26oZM5Q',
  'zyr8N8v8UDXhe8s5kaCRh4_T2J-et-HZ3VxqQZfKt3NEavxHmbAGKvU23Gtse4wrZf2GneblWy2KFQ',
  'jG-sRLtdoqVGEym1P4ASyuqx1e13Mb1QT8OqxBz1iqbvTqyRdpRYOcGzhC0-29qsqOPNd_ZRXuiw1w',
  'GVTQeUq3T8ClMUNJNTwrekNiL_89wfaUjjlFFQI2HvU5lK5j02X8mHLY8Ar-WzyLosVyaHv8yo5RWA']}

Metadata gives us particpants, looking at `player_data` this looks like they are `puuid`. So we could find a new match by selecting a random puuid and randomly selecting a match we haven't seen before simliar to the crawl idea in https://github.com/christophM/LolCrawler. Note I am repeating this github link because when we do write about the data collection process, we should cite this for giving us the idea behind it

In [141]:
player_data["puuid"], player_data["puuid"] in timeline["metadata"]["participants"]

('GVTQeUq3T8ClMUNJNTwrekNiL_89wfaUjjlFFQI2HvU5lK5j02X8mHLY8Ar-WzyLosVyaHv8yo5RWA',
 True)

Now for info

In [142]:
timeline["info"].keys()

dict_keys(['frameInterval', 'frames', 'gameId', 'participants'])

According to https://www.doranslab.gg/tools/jungle-tool.html, we get updates every minute, probably what below is saying

In [143]:
timeline["info"]["frameInterval"]

60000

So gameID gives us a new id type for the match, for partipants it looks like they have a unique id interally for the game... sure I guess that helps save space in the frames

In [144]:
timeline["info"]["gameId"], timeline["info"]["participants"]

(4913902543,
 [{'participantId': 1,
   'puuid': '_FznSp7W4096_tEkwFTiAmPJerwuyY9aqcMJ9CrvmeBIX59gZLPsMPb4iRdMV6DkUsIS6SeVBBc8Cw'},
  {'participantId': 2,
   'puuid': 'zhkSsv-0jE9dKvWh6h3xTO3UH6KIafKkI-v5E7zQjcKZWmnnzFyH415Ev-G0a1bKWo9o91IEKBumGQ'},
  {'participantId': 3,
   'puuid': '8ZwlZrtP-UWTq82MS34VbSvE3opxX6dD6AFwV0VUw29ovgRGwvSmarCxljvP1qzBSz6vmvXpFVAvxQ'},
  {'participantId': 4,
   'puuid': 'iyNh3kBlX5g3nTPqt03REuWavBhnySCx6Ims9-nIR5C9bOf2lJa--qFlOVp-OV-EsSrwPOVJXIEstA'},
  {'participantId': 5,
   'puuid': 'EgKQP7KwzGTALtoAkGdy4GOi1a9yNx_JqPqQJkWD75lKm80qFB6BmPL6-ebCb4Et6q1zK3o0hH8Y2w'},
  {'participantId': 6,
   'puuid': '0u5cOnC8z_gcVpDpG5OzbaiAF2-765y0yjZMseSrcHYkpg0aKV3n7rgAUG1nrHRV2E3NIYaHTa5pxg'},
  {'participantId': 7,
   'puuid': '-FYL436Q4NShi_V3LqLth92pviFCqpn3Gij_vA5dnscpBwJe4B7IJSixd3jV8uOrG_Hwbru26oZM5Q'},
  {'participantId': 8,
   'puuid': 'zyr8N8v8UDXhe8s5kaCRh4_T2J-et-HZ3VxqQZfKt3NEavxHmbAGKvU23Gtse4wrZf2GneblWy2KFQ'},
  {'participantId': 9,
   'puuid': 'jG-sRLt

Frames is where the data we care about lies, it is a list

In [145]:
type(timeline["info"]["frames"]), len(timeline["info"]["frames"])

(list, 19)

So above was around a 20 minute game, that explains alot I was playing aram. 

lets look at the frist point

In [146]:
timeline["info"]["frames"][0]

{'events': [{'realTimestamp': 1707196800096,
   'timestamp': 0,
   'type': 'PAUSE_END'}],
 'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
    'abilityPower': 0,
    'armor': 19,
    'armorPen': 0,
    'armorPenPercent': 0,
    'attackDamage': 25,
    'attackSpeed': 100,
    'bonusArmorPenPercent': 0,
    'bonusMagicPenPercent': 0,
    'ccReduction': 0,
    'cooldownReduction': 0,
    'health': 558,
    'healthMax': 558,
    'healthRegen': 0,
    'lifesteal': 0,
    'magicPen': 0,
    'magicPenPercent': 0,
    'magicResist': 30,
    'movementSpeed': 340,
    'omnivamp': 0,
    'physicalVamp': 0,
    'power': 385,
    'powerMax': 385,
    'powerRegen': 0,
    'spellVamp': 0},
   'currentGold': 1400,
   'damageStats': {'magicDamageDone': 0,
    'magicDamageDoneToChampions': 0,
    'magicDamageTaken': 0,
    'physicalDamageDone': 0,
    'physicalDamageDoneToChampions': 0,
    'physicalDamageTaken': 0,
    'totalDamageDone': 0,
    'totalDamageDoneToChampions': 0,
    'tot

In [147]:
timeline["info"]["frames"][0].keys()

dict_keys(['events', 'participantFrames', 'timestamp'])

In [148]:
timeline["info"]["frames"][0]["participantFrames"]["1"]

{'championStats': {'abilityHaste': 0,
  'abilityPower': 0,
  'armor': 19,
  'armorPen': 0,
  'armorPenPercent': 0,
  'attackDamage': 25,
  'attackSpeed': 100,
  'bonusArmorPenPercent': 0,
  'bonusMagicPenPercent': 0,
  'ccReduction': 0,
  'cooldownReduction': 0,
  'health': 558,
  'healthMax': 558,
  'healthRegen': 0,
  'lifesteal': 0,
  'magicPen': 0,
  'magicPenPercent': 0,
  'magicResist': 30,
  'movementSpeed': 340,
  'omnivamp': 0,
  'physicalVamp': 0,
  'power': 385,
  'powerMax': 385,
  'powerRegen': 0,
  'spellVamp': 0},
 'currentGold': 1400,
 'damageStats': {'magicDamageDone': 0,
  'magicDamageDoneToChampions': 0,
  'magicDamageTaken': 0,
  'physicalDamageDone': 0,
  'physicalDamageDoneToChampions': 0,
  'physicalDamageTaken': 0,
  'totalDamageDone': 0,
  'totalDamageDoneToChampions': 0,
  'totalDamageTaken': 0,
  'trueDamageDone': 0,
  'trueDamageDoneToChampions': 0,
  'trueDamageTaken': 0},
 'goldPerSecond': 0,
 'jungleMinionsKilled': 0,
 'level': 1,
 'minionsKilled': 0,
 'p

In [149]:
timeline["info"]["frames"][0]["participantFrames"]["1"].keys()

dict_keys(['championStats', 'currentGold', 'damageStats', 'goldPerSecond', 'jungleMinionsKilled', 'level', 'minionsKilled', 'participantId', 'position', 'timeEnemySpentControlled', 'totalGold', 'xp'])

Wait, that is really cool we get the following data:

['championStats', 'currentGold', 'damageStats', 'goldPerSecond', 'jungleMinionsKilled', 'level', 'minionsKilled', 'participantId', 'position', 'timeEnemySpentControlled', 'totalGold', 'xp']

And under chamption stats we get
['abilityHaste', 'abilityPower', 'armor', 'armorPen', 'armorPenPercent', 'attackDamage', 'attackSpeed', 'bonusArmorPenPercent', 'bonusMagicPenPercent', 'ccReduction', 'cooldownReduction', 'health', 'healthMax', 'healthRegen', 'lifesteal', 'magicPen', 'magicPenPercent', 'magicResist', 'movementSpeed', 'omnivamp', 'physicalVamp', 'power', 'powerMax', 'powerRegen', 'spellVamp']

Wow... that is a lot of data

In [150]:
timeline["info"]["frames"][0]["participantFrames"]["1"]["championStats"].keys()

dict_keys(['abilityHaste', 'abilityPower', 'armor', 'armorPen', 'armorPenPercent', 'attackDamage', 'attackSpeed', 'bonusArmorPenPercent', 'bonusMagicPenPercent', 'ccReduction', 'cooldownReduction', 'health', 'healthMax', 'healthRegen', 'lifesteal', 'magicPen', 'magicPenPercent', 'magicResist', 'movementSpeed', 'omnivamp', 'physicalVamp', 'power', 'powerMax', 'powerRegen', 'spellVamp'])

What does events actually mean?|

In [151]:
timeline["info"]["frames"][1]["events"]

[{'level': 2, 'participantId': 1, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 3, 'participantId': 1, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 2, 'participantId': 2, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 3, 'participantId': 2, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 2, 'participantId': 3, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 3, 'participantId': 3, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 2, 'participantId': 4, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 3, 'participantId': 4, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 2, 'participantId': 5, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 3, 'participantId': 5, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 2, 'participantId': 6, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 3, 'participantId': 6, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 2, 'participantId': 7, 'timestamp': 526, 'type': 'LEVEL_UP'},
 {'level': 3, 'participantId': 7, 'timestamp': 526,

### Okay that is interesting

So events tell us even more things that happened during that time. This is after the frist minute so naturally players are buying thier frist items.

In [152]:
timeline["info"]["frames"][2]["events"]

[{'itemId': 2003,
  'participantId': 6,
  'timestamp': 63354,
  'type': 'ITEM_DESTROYED'},
 {'itemId': 2003,
  'participantId': 1,
  'timestamp': 82120,
  'type': 'ITEM_DESTROYED'},
 {'level': 4, 'participantId': 1, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 2, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 3, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 4, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 5, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 6, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 7, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 8, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 9, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'level': 4, 'participantId': 10, 'timestamp': 84527, 'type': 'LEVEL_UP'},
 {'assistingParticipantIds': [6, 8, 9, 10],
  'bounty': 270,
  'kil

2 minutes in and we have a frist blood, and the position of that kill

{'assistingParticipantIds': [6, 8, 9, 10],
  'bounty': 270,
  'killStreakLength': 0,
  'killerId': 7,
  'position': {'x': 6906, 'y': 5944},
  'shutdownBounty': 0,
  'timestamp': 85594,
  'type': 'CHAMPION_KILL',
  'victimDamageDealt': .....

}

### Exploration done time for infrastructure

Goal: Create a dataframe of the current locations of each player for a single game shown above

In [153]:
from get_data import select_player_locations, get_next_puuid

data = select_player_locations(match_ids[0], "ARAM", timeline)
data

[{'match_ids': 'NA1_4913902543',
  'match_type': 'ARAM',
  'frame': 0,
  'player_part_id': '1',
  'player_puuid': '_FznSp7W4096_tEkwFTiAmPJerwuyY9aqcMJ9CrvmeBIX59gZLPsMPb4iRdMV6DkUsIS6SeVBBc8Cw',
  'x': 1096,
  'y': 1180,
  'level': 1},
 {'match_ids': 'NA1_4913902543',
  'match_type': 'ARAM',
  'frame': 0,
  'player_part_id': '2',
  'player_puuid': 'zhkSsv-0jE9dKvWh6h3xTO3UH6KIafKkI-v5E7zQjcKZWmnnzFyH415Ev-G0a1bKWo9o91IEKBumGQ',
  'x': 1135,
  'y': 1063,
  'level': 1},
 {'match_ids': 'NA1_4913902543',
  'match_type': 'ARAM',
  'frame': 0,
  'player_part_id': '3',
  'player_puuid': '8ZwlZrtP-UWTq82MS34VbSvE3opxX6dD6AFwV0VUw29ovgRGwvSmarCxljvP1qzBSz6vmvXpFVAvxQ',
  'x': 1099,
  'y': 944,
  'level': 1},
 {'match_ids': 'NA1_4913902543',
  'match_type': 'ARAM',
  'frame': 0,
  'player_part_id': '4',
  'player_puuid': 'iyNh3kBlX5g3nTPqt03REuWavBhnySCx6Ims9-nIR5C9bOf2lJa--qFlOVp-OV-EsSrwPOVJXIEstA',
  'x': 1000,
  'y': 870,
  'level': 1},
 {'match_ids': 'NA1_4913902543',
  'match_type': 'ARAM

In [154]:
get_next_puuid(player_data, timeline)

'zyr8N8v8UDXhe8s5kaCRh4_T2J-et-HZ3VxqQZfKt3NEavxHmbAGKvU23Gtse4wrZf2GneblWy2KFQ'

In [172]:
import random
def get_n_matches_data(lol_watcher, puuid, n, my_region="na1"):
    data = []
    for _ in range(n):
        match_ids = lol_watcher.match.matchlist_by_puuid(my_region, puuid)
        match_id = random.choice(match_ids)
        
        match_meta = lol_watcher.match.by_id(my_region, match_id)
        match_type = match_meta["info"]["gameMode"]

        timeline = lol_watcher.match.timeline_by_match(my_region, match_id)
        
        data = select_player_locations(match_id, match_type, timeline, data=data)
        puuid = get_next_puuid(puuid, timeline)
    return data

data = get_n_matches_data(lol_watcher, player_data['puuid'], 40, my_region="na1")



NA1_4906408509
NA1_4906497820
NA1_4913096250
NA1_4912102736
NA1_4910909165
NA1_4913902543
NA1_4912324242
NA1_4913986371
NA1_4913589399
NA1_4913547916
NA1_4912978472
NA1_4907133965
NA1_4912913660
NA1_4912913660
NA1_4912913660
NA1_4912853003
NA1_4914623258
NA1_4908870510
NA1_4914477445
NA1_4905350000
NA1_4878448614
NA1_4878626249
NA1_4914677064
NA1_4914689599
NA1_4914689599
NA1_4912961720
NA1_4912961720
NA1_4909800130
NA1_4910783748
NA1_4911830299
NA1_4911881634
NA1_4908226910
NA1_4913455426
NA1_4913647041
NA1_4913505349
NA1_4914589650
NA1_4913912584
NA1_4913952739
NA1_4913952739
NA1_4913500381


In [173]:
import pandas as pd

In [174]:
df = pd.DataFrame(data)
df

,match_ids,match_type,frame,player_part_id,player_puuid,x,y,level
0,NA1_4906408509,ARAM,0,1,kG7JSQpC7deUOSBnkqRxfOLB8nXC2sYwHLLUhDegbzKqq_...,1096,1180,1
1,NA1_4906408509,ARAM,0,2,ihzcXkONhaeuwgYCqIwhUmby-U5t4NXvvpKHnDAWdlJhNX...,1135,1063,1
2,NA1_4906408509,ARAM,0,3,89CjrTlPtYYoAjjEVSc-eVDAKk6aULd63RWKavHiIetyG8...,1099,944,1
3,NA1_4906408509,ARAM,0,4,pHkQUyo8Qpzk0s1ZPjF8GATX8pmBMZ62HmzgzII6Ys4k-A...,1000,870,1
4,NA1_4906408509,ARAM,0,5,-pN6bf-VQRe_f_G1kwF9IJB-D_z2GVWZwAmpf9Nar8ZAiD...,877,868,1
...,...,...,...,...,...,...,...,...
9935,NA1_4913500381,CLASSIC,33,6,5QCTz70pbl8vaygbTZ3cUsYMuifgg4YZyi5jyOins322BW...,3545,886,16
9936,NA1_4913500381,CLASSIC,33,7,lYyso47uG6N89fj9gCaeJuJiC96d5jXKcNhgO0jyddwgKw...,3756,938,16
9937,NA1_4913500381,CLASSIC,33,8,3WyuZHFN70XOgV1YF4_WrgKSdz_tboT-f6Cu4r068zt1-h...,1945,1995,17
9938,NA1_4913500381,CLASSIC,33,9,jdTqDr0j3DAY41RcS0gJMZIkil8mZQYSYv789TIxZojxqG...,2444,1036,16


In [175]:
df["player_puuid"].value_counts()

player_puuid
K2FCH6s02C0n8p8aUqxpGAZBFeYTgsHJ-Wrowg4S2IrdyMk6UpjTxKEVHdgQtuyyritlaJqaVyKDRw    281
7YT_7WOj--VM-hOxeDmoqR23aw17Nt9gJcDu8_VkCCw5-i7InktEEO1BoKwwHADM6eKB0KpOedLrsA    281
DSUPQEjZ_E5tW8rAQ02AiOiw44zBw2lgbFyNBmGU4_6wfp8Lqzye9dUd2pHOOBZpKL8NDm7QXtqyLQ    167
58567Yu693KRwzpLFz6-NYhL3lZpKFMcog4h-zVgBJ9qMO8JpxyuH8r1PNCjmrawvtssJOtKKIERTg    145
jdTqDr0j3DAY41RcS0gJMZIkil8mZQYSYv789TIxZojxqGCV5abSRR6V3hRhY7blRvWka-xpeseOdA    124
                                                                                 ... 
25jjH-PbdqrRKaYfOY9M423yrqdhI_vvvA2D00mferXoDt-OZru2BpZUqyThE1iwUVQuchVur26DWg      3
-AhLouJhxC-vkrERd1d6euuCAlAccVOcTuY56B53gD0AcaozHOTIMRd997qy27yXqJ5VYh_AcYH2XA      3
G7f1XDpeiY_sn51S5g1nHeWmwGIQ1B0p13V_zQpKKC1JX1-Sv1MVuwNV_gx9BT6sfVWkv55ecjY7EQ      3
NmdFPuXLz3U99fWtgGBA5qw44QS7w3VNgxswyjy3GZlUVHNwdq9v2607O6wBzcDnPcAMN1YqfPjmew      3
6iYYg63rihDnUopHKh5XimNh6yw4RB9GYlwaq-4MimGPTe6D2VIekzTNUmDaBqZ0SKiwTjAop2yMSA      3
Name: count, Length: 286, dtype: int64

In [176]:
df["match_ids"].value_counts()

match_ids
NA1_4912913660    1110
NA1_4913952739     900
NA1_4907133965     470
NA1_4914689599     400
NA1_4908226910     370
NA1_4908870510     370
NA1_4913500381     340
NA1_4912978472     340
NA1_4913912584     310
NA1_4914589650     310
NA1_4914623258     300
NA1_4912961720     280
NA1_4912102736     270
NA1_4913589399     270
NA1_4905350000     250
NA1_4910909165     240
NA1_4914477445     240
NA1_4913547916     230
NA1_4909800130     230
NA1_4912853003     220
NA1_4906408509     220
NA1_4912324242     200
NA1_4913647041     200
NA1_4913096250     200
NA1_4911830299     200
NA1_4913902543     190
NA1_4913455426     180
NA1_4878626249     170
NA1_4910783748     170
NA1_4913986371     150
NA1_4878448614     150
NA1_4911881634     150
NA1_4914677064     140
NA1_4913505349     140
NA1_4906497820      30
Name: count, dtype: int64

In [177]:
df["match_type"].value_counts()

match_type
ARAM       6570
CLASSIC    3370
Name: count, dtype: int64

In [179]:
df["match_ids"].nunique()

35